In [1]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torchtext.datasets.imdb import IMDB
import sys

In [2]:
torch.__version__
sys.version

'3.7.2 (default, Dec 29 2018, 06:19:36) \n[GCC 7.3.0]'

In [3]:
sys.getdefaultencoding()

'utf-8'

In [4]:
is_cuda = False

if torch.cuda.is_available():
    is_cuda=True


In [5]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

In [6]:
train, test = IMDB.splits(TEXT, LABEL)

In [7]:
type(train)

torchtext.datasets.imdb.IMDB

In [8]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

train.fields {'text': <torchtext.data.field.Field object at 0x7f99d12d2ef0>, 'label': <torchtext.data.field.Field object at 0x7f99d12d2f60>}
len(train) 25000
vars(train[0]) {'text': ['girlfight', 'is', 'like', 'your', "grandmother's", 'cooking:', 'same', 'old', 'recipe', "you've", 'tried', 'a', 'million', 'times', 'before,', 'yet', 'somehow', 'transformed', 'into', 'something', 'fresh', 'and', 'new.', 'try', 'and', 'explain', 'the', 'story', 'to', 'people', 'who', "haven't", 'seen', 'it', 'before:', 'a', 'young', 'women', 'from', 'the', 'wrong', 'side', 'of', 'the', 'tracks', 'attempts', 'to', 'improve', 'her', 'situation', 'by', 'taking', 'up', 'boxing', 'whilst', 'dealing', 'with', 'a', 'bitter,', 'obstructive', 'father', 'and', 'her', 'growing', 'attraction', 'to', 'a', 'male', 'rival.', 'watch', 'them', 'roll', 'their', 'eyes', 'at', 'the', 'string', 'of', 'clichés,', 'and', "they're", 'right:', 'it', '*is*', 'clichéd.', 'yet', 'i', 'was', 'hypnotized', 'by', 'how', 'well', 'this',

In [9]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [10]:
LABEL.vocab.freqs

Counter({'pos': 12500, 'neg': 12500})

In [11]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

train.fields {'text': <torchtext.data.field.Field object at 0x7f99d12d2ef0>, 'label': <torchtext.data.field.Field object at 0x7f99d12d2f60>}
len(train) 25000
vars(train[0]) {'text': ['girlfight', 'is', 'like', 'your', "grandmother's", 'cooking:', 'same', 'old', 'recipe', "you've", 'tried', 'a', 'million', 'times', 'before,', 'yet', 'somehow', 'transformed', 'into', 'something', 'fresh', 'and', 'new.', 'try', 'and', 'explain', 'the', 'story', 'to', 'people', 'who', "haven't", 'seen', 'it', 'before:', 'a', 'young', 'women', 'from', 'the', 'wrong', 'side', 'of', 'the', 'tracks', 'attempts', 'to', 'improve', 'her', 'situation', 'by', 'taking', 'up', 'boxing', 'whilst', 'dealing', 'with', 'a', 'bitter,', 'obstructive', 'father', 'and', 'her', 'growing', 'attraction', 'to', 'a', 'male', 'rival.', 'watch', 'them', 'roll', 'their', 'eyes', 'at', 'the', 'string', 'of', 'clichés,', 'and', "they're", 'right:', 'it', '*is*', 'clichéd.', 'yet', 'i', 'was', 'hypnotized', 'by', 'how', 'well', 'this',

In [12]:
d = vars(TEXT.vocab)

In [13]:
d.keys()

dict_keys(['freqs', 'itos', 'stoi', 'vectors'])

In [14]:
TEXT.vocab.vectors



tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7724, -0.1800,  0.2072,  ...,  0.6736,  0.2263, -0.2919],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [15]:
len(TEXT.vocab.stoi)

10002

In [16]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1)

train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [17]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc = nn.Linear(hidden_size2,3)
        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc(embeds)
        return F.log_softmax(out,dim=-1)
        
        

In [18]:
model = EmbNet(len(TEXT.vocab.stoi),10)
model = model.cuda()

In [19]:

optimizer = optim.Adam(model.parameters(),lr=0.001)

In [20]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [21]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [22]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

In [23]:
%%time
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

/home/taewan/.local/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training loss is  0.73 and training accuracy is 13009/25000     52.04
validation loss is   0.7 and validation accuracy is 13360/25000     53.44
training loss is  0.68 and training accuracy is 14420/25000     57.68
validation loss is  0.68 and validation accuracy is 14590/25000     58.36
training loss is  0.64 and training accuracy is 15903/25000     63.61
validation loss is  0.65 and validation accuracy is 15828/25000     63.31
training loss is  0.59 and training accuracy is 17104/25000     68.42
validation loss is  0.62 and validation accuracy is 16442/25000     65.77
training loss is  0.54 and training accuracy is 18078/25000     72.31
validation loss is  0.62 and validation accuracy is 16888/25000     67.55
training loss is  0.51 and training accuracy is 18664/25000     74.66
validation loss is  0.61 and validation accuracy is 17158/25000     68.63
training loss is  0.48 and training accuracy is 19176/25000      76.7
validation loss is   0.6 and validation accuracy is 17435/25000   

## 사전 학습 Glove 워드 임베딩 사용하기

In [24]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

train, test = IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train,test, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [25]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc1 = nn.Linear(hidden_size2,3)

        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc1(embeds)
        return F.log_softmax(out,dim=-1)

In [26]:
model = EmbNet(len(TEXT.vocab.stoi),300,12000)
model = model.cuda()

In [27]:
model.embedding.weight.data = TEXT.vocab.vectors.cuda()

In [28]:
model.embedding.weight.requires_grad = False

In [29]:
#optimizer = optim.SGD(model.parameters(),lr=0.001)
optimizer = optim.Adam([ param for param in model.parameters() if param.requires_grad == True],lr=0.001)

In [30]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=64, device=-1,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [31]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [32]:
%%time
for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.66 and training accuracy is 15714/25000     62.86
validation loss is  0.65 and validation accuracy is 16218/25000     64.87
training loss is  0.57 and training accuracy is 17674/25000      70.7
validation loss is  0.65 and validation accuracy is 16555/25000     66.22
training loss is  0.53 and training accuracy is 18425/25000      73.7
validation loss is  0.67 and validation accuracy is 16425/25000      65.7
training loss is  0.51 and training accuracy is 18779/25000     75.12
validation loss is  0.69 and validation accuracy is 16358/25000     65.43
training loss is  0.49 and training accuracy is 19145/25000     76.58
validation loss is  0.72 and validation accuracy is 16375/25000      65.5
training loss is  0.48 and training accuracy is 19169/25000     76.68
validation loss is  0.74 and validation accuracy is 16361/25000     65.44
training loss is  0.47 and training accuracy is 19435/25000     77.74
validation loss is  0.76 and validation accuracy is 16207/25000   

In [33]:
%%time
for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.44 and training accuracy is 19858/25000     79.43
validation loss is   0.8 and validation accuracy is 16197/25000     64.79
training loss is  0.44 and training accuracy is 19788/25000     79.15
validation loss is  0.82 and validation accuracy is 16121/25000     64.48
training loss is  0.44 and training accuracy is 19901/25000      79.6
validation loss is  0.83 and validation accuracy is 16113/25000     64.45
training loss is  0.43 and training accuracy is 20016/25000     80.06
validation loss is  0.85 and validation accuracy is 16093/25000     64.37
training loss is  0.43 and training accuracy is 20029/25000     80.12
validation loss is  0.87 and validation accuracy is 15947/25000     63.79
training loss is  0.42 and training accuracy is 20139/25000     80.56
validation loss is  0.89 and validation accuracy is 15935/25000     63.74
training loss is  0.42 and training accuracy is 20145/25000     80.58
validation loss is  0.88 and validation accuracy is 16042/25000   